In [ ]:
from cortexlib.utils.file import write_json_file, save_filtered_neural_data, save_model_features, Model
from cortexlib.mouse import CortexlabMouse
from cortexlib.images import CortexlabImages
from cortexlib.vgg19 import PreTrainedVGG19Model
from cortexlib.predictor import NeuralResponsePredictor
from cortexlib.utils.plotting import vgg_colours
from cortexlib.utils.random import set_global_seed
from cortexlib.utils.logging import Logger
import matplotlib.pyplot as plt
import pandas as pd

logger = Logger()
set_global_seed()

In [2]:
mouse = CortexlabMouse()

logger.progress("Computing null distributions for all neurons")
null_srv_all_neurons = mouse.compute_null_all_neurons(n_shuffles=100)
logger.success(f"Null distributions computed")

real_srv_all_neurons = mouse.compute_real_srv_all_neurons()
reliable_neuron_indices = mouse.get_reliable_neuron_indices(
            null_srv_all_neurons, real_srv_all_neurons, percentile_threshold=99)
neural_responses_mean, neural_responses, _ = mouse.get_responses_for_reliable_neurons(reliable_neuron_indices, real_srv_all_neurons, num_neurons=500)

logger.info(f"Neural responses shape: {neural_responses.shape}")

19:50:09 | INFO     | ⏳ Computing null distributions for all neurons...
19:50:31 | SUCCESS  | ✅ Null distributions computed!
19:50:31 | INFO     | ℹ️ Neural responses shape: (1573, 2, 500)


In [3]:
save_filtered_neural_data(
    mouse_id='1', # TODO: use real mouse.id
    neural_responses=neural_responses,
    neural_responses_mean=neural_responses_mean)

19:50:32 | INFO     | ℹ️ Skipping save, file already exists at /Users/callummessiter/workspace/msc-neuro/research-project/analysis/mouse_1/_neural_data/neural_data_mouse_1.pt


In [ ]:
vgg19 = PreTrainedVGG19Model()

settings = vgg19.get_image_settings()
images = CortexlabImages(
    size=settings['size'],
    channels=settings['channels'],
    normalise_mean=settings['mean'], 
    normalise_std=settings['std'],
    rescale_per_image=settings['rescale_per_image'],
)

logger.progress("Loading and preprocessing images shown to mouse")
image_dataset = images.load_images_shown_to_mouse(mouse.image_ids)
logger.success("Images processed")

19:50:32 | INFO     | ℹ️ Already downloaded pretrained SimCLR model
19:50:32 | INFO     | ⏳ Loading and preprocessing images shown to mouse...
19:50:47 | SUCCESS  | ✅ Images processed!


In [ ]:
logger.progress("VGG19: extracting features from images shown to mouse")
vgg19_feats, labels = vgg19.extract_features(image_dataset)
logger.success("VGG19 features extracted")

for layer, feats in vgg19_feats.items():
    logger.info(f"{layer} feats shape: {tuple(feats.shape)}")

19:50:47 | INFO     | ⏳ SimCLR: extracting features from images shown to mouse...


  0%|          | 0/25 [00:00<?, ?it/s]

19:51:04 | SUCCESS  | ✅ SimCLR features extracted!
19:51:04 | INFO     | ℹ️ layer1 feats shape: (1573, 64, 24, 24)
19:51:04 | INFO     | ℹ️ layer2 feats shape: (1573, 128, 12, 12)
19:51:04 | INFO     | ℹ️ layer3 feats shape: (1573, 256, 6, 6)
19:51:04 | INFO     | ℹ️ layer4 feats shape: (1573, 512, 3, 3)
19:51:04 | INFO     | ℹ️ fc feats shape: (1573, 512)


In [ ]:
save_model_features(model=Model.VGG19, mouse_id="1", features=vgg19_feats, labels=labels) # TODO: use real mouse.id

19:51:04 | INFO     | ℹ️ Skipping save, file already exists at /Users/callummessiter/workspace/msc-neuro/research-project/analysis/mouse_1/_model_features/simclr_features_mouse_1.pt


In [ ]:
fev_results = []

logger.progress("Running regressions: VGG19 Feats => PC1 Neural Data")
for layer, feats in vgg19_feats.items():
    logger.progress(f"VGG19 {layer}")
    
    for n_pcs in [None, 10, 20, 50, 100, 200, 300, 400, 500]:
        predictor = NeuralResponsePredictor(reduce_image_representation_to_n_pcs=n_pcs, neural_data_pc_index=0)
        r_squared = predictor.compute_r_squared(feats, neural_responses)
        fev = predictor.compute_fev(feats, neural_responses)

        fev_results.append({
            'layer': layer,
            'n_pcs': n_pcs,
            'test_r2': r_squared['test_r2'],
            'mean_fev': fev['mean_fev']
        })

logger.success(f"Regressions complete")

19:51:04 | INFO     | ⏳ Running regressions: SimCLR Feats => PC1 Neural Data...
19:51:04 | INFO     | ⏳ SimCLR layer1...


In [ ]:
write_json_file(fev_results, f'./prediction.json')

In [ ]:
fev_results_df = pd.DataFrame(fev_results)
fev_results_df.head(12)

In [ ]:
plt.figure(figsize=(10, 6))

for layer in fev_results_df['layer'].unique():
    layer_results = fev_results_df[fev_results_df['layer'] == layer]
    plt.plot(layer_results['n_pcs'], layer_results['mean_fev'], marker='o', color=vgg19_colours[layer], label=layer)

plt.xlabel('Number of Principal Components')
plt.ylabel('Mean FEV')
plt.title('Mean FEV vs Number of PCs for VGG19 Layers')
plt.legend(title='Layer')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
best_fev_per_layer = fev_results_df.loc[fev_results_df.groupby("layer")["mean_fev"].idxmax()]
best_fev_per_layer

In [ ]:
desired_order = vgg19_feats.keys()
best_fev_per_layer = best_fev_per_layer.set_index("layer").loc[desired_order].reset_index()
layers = [row["layer"] for _, row in best_fev_per_layer.iterrows()]
mean_fevs = [row["mean_fev"] for _, row in best_fev_per_layer.iterrows()]

colors = [vgg19_colours[layer] for layer in layers]

plt.figure(figsize=(8, 5))
plt.bar(layers, mean_fevs, color=colors)
plt.ylabel("Mean FEV")
plt.xlabel("VGG19 Layer")
plt.title("Raw Neural Data: Mean FEV per Layer")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()